## Add A Random Noise 

For this example, we consider a Gaussian noise centered at $0$ of `std_dev = 5`= at each evaluation point of $f$. 

In [ ]:
using Globtim
using DynamicPolynomials, DataFrames
using PlotlyJS, Colors
using Distributions

include("../../src/lib_func.jl")

# Constants and Parameters
d = 1 # Initial Degree 
const n, a, b = 2, 11, 10 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters # Delta used to be too big 
const tol_l2 = 1e-4            # Define the tolerance for the L2-norm
const sample_scale = 1.0

function noisy_Deuflhard(xx::Vector{Float64}; mean::Float64=0.0, stddev::Float64=5.0)::Float64
    noise = rand(Normal(mean, stddev))
    return Deuflhard(xx) + noise
end

f_noisy = noisy_Deuflhard
d = 1
noisy_tol_l2 = 2.6e-2        # Define the noise affected tolerance for the L2-norm
alpha_noise = 0.20 # probability of discrete L2 norm not being accurate to the set tolerance.

while true # Potential infinite loop
    global poly_approx_noisy = MainGenerate(f_noisy, 2, d, delta, alpha_noise, scale_factor, sample_scale) # computes the approximant in Chebyshev basis
    if poly_approx_noisy.nrm < noisy_tol_l2
        println("attained the desired L2-norm: ", poly_approx_noisy.nrm)
        println("Degree: $d")
        break
    else
        println("current L2-norm: ", poly_approx_noisy.nrm)
        println("Number of samples: ", poly_approx_noisy.N)
        global d += 1
    end
end
# ==================================================================================================
loc = "inputs.ms"
# File path of the output file
file_path_output = "outputs.ms";
# ==================================================================================================
ap = main_nd(n, d, poly_approx_noisy.coeffs)
# ==================================================================================================
@polyvar(x[1:n]) # Define polynomial ring 
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x
names = [x[i].name for i in 1:length(x)]
open(loc, "w") do file
    println(file, join(names, ", "))
    println(file, 0)
end
# Define the polynomial approximant 
PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
for i in 1:n
    partial = differentiate(PolynomialApproximant, x[i])
    partial_str = replace(string(partial), "//" => "/")
    open(loc, "a") do file
        if i < n
            println(file, string(partial_str, ","))
        else
            println(file, partial_str)
        end
    end
end
# ==================================================================================================
run(`msolve -v 0 -f inputs.ms -o outputs.ms`)
# ==================================================================================================
evaled = process_output_file(file_path_output)  # Process the file and get the points

real_pts = []
for pts in evaled
    if typeof(pts) == Vector{Vector{Vector{BigInt}}}
        X = parse_point(pts) # Parse the points into correct format
    else
        X = average.(pts)
    end
    push!(real_pts, Float64.(X))
end

# ==================================================================================================
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector

# Here we should evaluate on the noiseless function to compare with previous results
h_z = map(p -> Deuflhard([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))

df_noisy = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

coords = poly_approx_noisy.scale_factor * poly_approx_noisy.grid
z_coords = poly_approx_noisy.z

# ==================================================================================================
# Plot the 3D scatter plot if the dimensions are 2
if size(coords)[2] == 2
    scatter_trace = scatter3d(
        x=coords[:, 1],
        y=coords[:, 2],
        z=z_coords,
        mode="markers",
        marker=attr(
            size=1,
            color=z_coords,
            colorscale="Viridis"
        ),
        name="Sampled Noisy Data"
    )
    # Had to switch the coordinates of the critical points to match the surface plot for some reason. 
    crit_pts_noisy = scatter3d(
        x=df_noisy.y,
        y=df_noisy.x,
        z=df_noisy.z,
        mode="markers",
        marker=attr(
            size=8,
            color="orange"
        ),
        name="Critical Points"
    )

    layout = Layout(
        title="3D Scatter Plot of Sample Points",
        scene=attr(
            xaxis=attr(title="X-axis"),
            yaxis=attr(title="Y-axis"),
            zaxis=attr(title="Z-axis")),
        height=800
    )
end

In [ ]:
println("L2 tolerance: $noisy_tol_l2")
println("Degree: $d")
println("current L2-norm: ", poly_approx_noisy.nrm)
println("Number of samples: ", poly_approx_noisy.N)
plt = Plot([scatter_trace, crit_pts_noisy], layout)

### We make the tolerance tighter

In [ ]:
noisy_tol_l2 = 2.2e-2
d = 1
while true # Potential infinite loop
    global poly_approx_noisy = MainGenerate(f_noisy, 2, d, delta, alpha_noise, scale_factor, sample_scale) # computes the approximant in Chebyshev basis
    if poly_approx_noisy.nrm < noisy_tol_l2
        println("attained the desired L2-norm: ", poly_approx_noisy.nrm)
        println("Degree: $d")
        break
    else
        println("current L2-norm: ", poly_approx_noisy.nrm)
        println("Number of samples: ", poly_approx_noisy.N)
        global d += 1
    end
end
ap = main_nd(n, d, poly_approx_noisy.coeffs)
@polyvar(x[1:n]) # Define polynomial ring 
names = [x[i].name for i in 1:length(x)]
open(loc, "w") do file
    println(file, join(names, ", "))
    println(file, 0)
end
PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
for i in 1:n
    partial = differentiate(PolynomialApproximant, x[i])
    partial_str = replace(string(partial), "//" => "/")
    open(loc, "a") do file
        if i < n
            println(file, string(partial_str, ","))
        else
            println(file, partial_str)
        end
    end
end
run(`msolve -v 1 -f inputs.ms -o outputs.ms`)
evaled = process_output_file(file_path_output)
real_pts = []
for pts in evaled
    if typeof(pts) == Vector{Vector{Vector{BigInt}}}
        X = parse_point(pts)
    else
        X = average.(pts)
    end
    push!(real_pts, Float64.(X))
end
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> Deuflhard([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df_5_noisy = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

## For Plots ##
# Collect the Sample set and evaluations data and rescale it 
coords = poly_approx_noisy.scale_factor * poly_approx_noisy.grid
z_coords = poly_approx_noisy.z

# Plot the 3D scatter plot if the dimensions are 2
scatter_trace = scatter3d(
    x=coords[:, 1],
    y=coords[:, 2],
    z=z_coords,
    mode="markers",
    marker=attr(
        size=1,
        color=z_coords,
        colorscale="Viridis"
    ),
    name="Sampled Noisy Data"
)

crit_pts_5_noisy = scatter3d(
    x=df_5_noisy.y,
    y=df_5_noisy.x,
    z=df_5_noisy.z,
    mode="markers",
    marker=attr(
        size=8,
        color="orange"
    ),
    name="Critical Points"
)


In [ ]:
println("L2 tolerance: $noisy_tol_l2")
println("Degree: $d")
println("current L2-norm: ", poly_approx_noisy.nrm)
println("Number of samples: ", poly_approx_noisy.N)
plt = Plot([scatter_trace, crit_pts_5_noisy], layout)